In [ ]:
%matplotlib inline

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15,8

In [ ]:
import scipy.stats
import analysis
from matplotlib import pyplot
from matplotlib import colors

In [ ]:
data = analysis.get_full_dict()

In [ ]:
correlations = {}

any = next(iter(data.keys()))
questions = list(data[any].keys())
top_pvals = []

for i in range(len(questions)):
    row = questions[i]
    x = analysis.pull_data(data, data.keys(), row)

    if not isinstance(x[0], (int, float)):
        continue
        
    correlations[row] = {}
    
    for j in range(len(questions)):
        col = questions[j]
        y = analysis.pull_data(data, data.keys(), col)
        
        if not isinstance(y[0], (int, float)):
            continue
        
        correlations[row][col] = scipy.stats.pearsonr(x, y)
        
        if j < i:
            top_pvals.append((row, col) + correlations[row][col])

In [ ]:
sorted(top_pvals, key=lambda x: x[3])[:5]

In [ ]:
matrix = []
keys = []

for k in ('usability', 'efficiency', 'knowledge'):
    keys += analysis.qs_entry[k]
    
for k in ('usability', 'education', 'intention'):
    keys += analysis.qs_exit[k]

for i, row in enumerate(keys):
    matrix.append([])
    for j, col in enumerate(keys):
        matrix[i].append(correlations[row][col])

In [ ]:
my_cmap = colors.LinearSegmentedColormap(
    'Custom',
    {
        'red':   ((0.0, 1.0, 1.0),
                  (0.4, 1.0, 1.0),
                  (0.8, 0.0, 0.0),
                  (1.0, 1.0, 1.0)),

        'green': ((0.0, 0.0, 0.0),
                  (0.4, 1.0, 1.0),
                  (0.8, 0.0, 0.0),
                  (1.0, 1.0, 1.0)),

        'blue':  ((0.0, 0.0, 0.0),
                  (0.4, 1.0, 1.0),
                  (0.8, 1.0, 1.0),
                  (1.0, 1.0, 1.0))
    }
)

fig, ax = pyplot.subplots()
image = [[(1 + x[0]) / 2.5 if x[1] < 0.05 else 1.0 for x in row] for row in matrix]
ax.imshow(image, cmap=my_cmap, interpolation='nearest')
pyplot.xlabel("Signifigant Correlations", fontsize=16)
ax.xaxis.tick_top()
pyplot.xticks(range(len(keys)), keys, rotation='vertical')
pyplot.yticks(range(len(keys)), keys)
#pyplot.setp(pyplot.xticks()[1], rotation=270)

fig, ax = pyplot.subplots()
image = [[1 - 20 * x[1] if x[1] < 0.05 else 0 for x in row] for row in matrix]
ax.imshow(image, cmap=pyplot.cm.Blues, interpolation='nearest')
pyplot.title("Signifigant P-Values", fontsize=16)